In [1]:
import os
import sys
import csv
import pickle
import random
import math
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.parametrizations import spectral_norm

from torchtext.vocab import build_vocab_from_iterator

from sklearn.metrics import roc_auc_score

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, 'r') as csvfile:
        train_data = list(csv.reader(csvfile))[1:] # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

# number of trainable parameters in model
def get_total_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [5]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class CleavageBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def collate_wrapper(batch):
    return CleavageBatch(batch)

In [6]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_size1, rnn_size2, hidden_size, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
        )
        
        self.dropout=nn.Dropout(dropout)
        
        self.lstm1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=rnn_size1,
            bidirectional=True,
            batch_first=True,
        )

        self.lstm2 = nn.LSTM(
            input_size=2 * rnn_size1,
            hidden_size=rnn_size2,
            bidirectional=True,
            batch_first=True,
        )
        
        self.fc1 = spectral_norm(nn.Linear(rnn_size2 * 2, hidden_size))
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, seq):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.embedding(seq))
        
        # input shape: (batch_size, seq_len, embedding_dim)
        out, _ = self.lstm1(embedded)
        
        # input shape: (batch_size, seq_len, 2*rnn_size1)
        out, _ = self.lstm2(out)
        
        # input shape: (batch_size, seq_len, 2*hidden_size)
        pooled, _ = torch.max(out, dim=1)
        
        # input shape: (batch_size, 2*hidden_size)
        out = self.dropout(gelu(self.fc1(pooled)))
        
        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [7]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += len(seq)
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../../data/n_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [9]:
NUM_EPOCHS = 25
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 200
RNN_SIZE1 = 256
RNN_SIZE2 = 512
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 5e-5

model = BiLSTM(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_size1=RNN_SIZE1,
    rnn_size2=RNN_SIZE2,
    hidden_size=HIDDEN_SIZE,
    dropout=DROPOUT
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

print(f"Total trainable model parameters: {get_total_model_params(model):,}")

Total trainable model parameters: 5,276,009


In [10]:
start = time()
print("Starting Training.")
highest_val_auc = 0
train_losses, train_accuracies, train_aucs = [], [], []
val_losses, val_accuracies, val_aucs = [], [], []

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss, train_acc, train_auc = process(model, train_loader, criterion, optimizer)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_acc, val_auc = process(model, dev_loader, criterion)
        
    # save current acc, loss
    train_losses.append((epoch, train_loss))
    train_accuracies.append((epoch, train_acc))
    train_aucs.append((epoch, train_auc))
    val_losses.append((epoch, val_loss))
    val_accuracies.append((epoch, val_acc))
    val_aucs.append((epoch, val_auc))
    
    if val_auc > highest_val_auc:
        highest_val_auc = val_auc
        path = f"../../params/n_term/BiLSTM/auc{val_auc:.4f}_epoch{epoch}.pt"
        torch.save(model.state_dict(), path)
        
    print(
        f"Training:   [Epoch {epoch:2d}, Loss: {train_loss:8.4f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Loss: {val_loss:8.4f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}]")
    
print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 305.26batches/s]
Training:   [Epoch  1, Loss:   0.0013, Acc: 0.6187, AUC: 0.6540]
Evaluation: [Epoch  1, Loss:   0.0011, Acc: 0.6825, AUC: 0.7499]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 319.75batches/s]
Training:   [Epoch  2, Loss:   0.0011, Acc: 0.6848, AUC: 0.7513]
Evaluation: [Epoch  2, Loss:   0.0011, Acc: 0.6925, AUC: 0.7632]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 306.15batches/s]
Training:   [Epoch  3, Loss:   0.0011, Acc: 0.6900, AUC: 0.7585]
Evaluation: [Epoch  3, Loss:   0.0011, Acc: 0.6921, AUC: 0.7652]
Eval: 100%|██████████████████████████████████████████████████████| 280/280 [00:00<00:00, 312.81batches/s]
Training:   [Epoch  4, Loss:   0.0011, Acc: 0.6913, AUC: 0.7610]
Evaluation: [Epoch  4, Loss:   0.0011, Acc: 0.6933, AUC: 0.7661]
Eval: 100%|██████████████████████████

In [ ]:
# save training stats
lsts = [train_losses, train_accuracies, val_losses, val_accuracies, train_aucs, val_aucs, train_time]
names = [
    "train_losses",
    "train_accuracies",
    "val_losses",
    "val_accuracies",
    "train_aucs",
    "val_aucs",
    "train_time",
]
to_save = dict()
for name, lst in zip(names, lsts):
    to_save[name] = lst

with open(f"../../params/n_term/BiLSTM/metrics.pkl", "wb") as f:
    pickle.dump(to_save, f, pickle.HIGHEST_PROTOCOL)

print("Finished Saving Details.")